This Notebook is based on the work by [Hank Rearden](https://www.kaggle.com/hreard) in [2 Hidden Layers w/ ReLU Activation NN](https://www.kaggle.com/hreard/2-hidden-layers-w-relu-activation-nn)

# Import

In [ ]:
import numpy as np
import pandas as pd
import os

from sklearn.model_selection import train_test_split

import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelBinarizer
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor

import timeit

# Load Data

In [ ]:
df0 = pd.read_csv('../input/emg-4/0.csv', header = None)
df1 = pd.read_csv('../input/emg-4/1.csv', header = None)
df2 = pd.read_csv('../input/emg-4/2.csv', header = None)
df3 = pd.read_csv('../input/emg-4/3.csv', header = None)

In [ ]:
df = pd.concat([df0, df1, df2, df3])

In [ ]:
df[64] = df[64].astype('category')

In [ ]:
Y = df[64]
X = df.drop(columns=64)

n_cols = len(X.columns)

In [ ]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size = 0.2,random_state=42)   

Targets are ordinal, but since there is no implicit order between the different gestures (e.g gesture 1 isn't between 0 and 2) we will use a one-hot encoding for the target variable.

In [ ]:
enc = LabelBinarizer()

In [ ]:
Ytrain = enc.fit_transform(Ytrain)
Ytest = enc.transform(Ytest)

# Model

In [ ]:
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(34, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(17, activation='relu'))
    model.add(Dense(4, activation='softmax'))
    
    # compile model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

We will use categorical_crossentropy since we convert the target variable to a one-hot encoding. If we kept the variable ordinal we would use sparse_categorical_crossentropy.

In [ ]:
clf = KerasRegressor(build_fn=create_model) # There is an issue with kerasClassifier at this time, it still uses predict_classes which is deprecated. To avoid this we use KerasRegressor.

pipe = Pipeline([('scaler', StandardScaler()), ('clf', clf)])

# Training

In [ ]:
pipe.fit(Xtrain,Ytrain, clf__epochs=100, clf__verbose=0)

# Evaluate

In [ ]:
pipe.score(Xtrain,Ytrain)

In [ ]:
pipe.score(Xtest,Ytest)

# Inference time

In [ ]:
preds_test = []
time_test = []
for i in range(Xtest.shape[0]):
  x = Xtest.iloc[[i]]
  start = timeit.default_timer()
  pred = pipe.predict(x)
  stop = timeit.default_timer()
  preds_test.append(pred)
  time_test.append(stop - start)
np.mean(time_test)

# Save pipeline

In [ ]:
import joblib
# joblib.dump(pipe, 'pipeline') # causes an error "can't pickle weakref objects" needs fix